In [1]:
import numpy as np
import viscofit as vf

# Load IBW Files

In [4]:
###### THIS IS PLACE HOLDER, MUST LOAD AFM DATA
times = [np.linspace(0, 1, 1000) for i in range(3)]
indentations = [(i + 1) * 100e-9 * t / max(t) for i, t in enumerate(times)]
radii = [(i + 1) * 50e-9 for i in range(len(times))]
forces = [vf.forceMaxwell_LeeRadok(np.array([1e5, 1e6, 1e-3]), t, h, r) for t, h, r in zip(times, indentations, radii)]

# Fit Standard Models

### ----------------
### Maxwell Fit
### ----------------

### maxwellModel(forces, times, indentations, radii, E_logbounds=(1, 9), T_logbounds=(-5, 0)

##### initializes an instance of the maxwellModel class used for generating fits, of experimentally obtained force-distance data all belonging to the same sample, to a maxwell model which corresponds to the sample's viscoelastic behavior
##### :param forces: either list of numpy arrays or single numpy array corresponding to the force signals from an AFM
##### :param times: either list of numpy arrays or single numpy array corresponding to the time signals from an AFM
##### :param indentations: either list of numpy arrays or single numpy array corresponding to the indentation signals from an AFM
##### :param radii: either list of floats or single float corresponding to the tip radii of an AFM
##### :param E_logbounds: tuple (float, float) high and low log bound for the elastic elements in the model
##### :param T_logbounds: tuple (float, float) high and low log bound for the time constants in the model

In [9]:
# create a maxwellModel instance
# load the AFM forces, times, indentations, and radii
# define the logbounds for E to be between 4 and 7 -> (10^4, 10^7)
# define the logbounds for T to be between -4 and -2 -> (10^-4, 10^-2)
model1 = vf.maxwellModel(forces=forces, times=times, indentations=indentations, 
                         radii=radii, E_logbounds=(4, 7), T_logbounds=(-4, -2))

### maxwellModel.fit(maxiter=1000, max_model_size=4, fit_sequential=True, num_attempts=5)

##### fit experimental force distance curve(s) to maxwell model of arbitrary size using a nelder-mead simplex which typically gives good fits rather quickly
##### :param maxiter: int maximum iterations to perform for each fitting attempt (larger number gives longer run time)
##### :param max_model_size: int largest number of arms per maxwell model to test (going larger tends to give poor and unphysical fits)
##### :param fit_sequential: bool whether or not to fit sequentially (cascade fit from previous model as the initial guess of the next) (RECOMMENDED)
##### :param num_attempts: int number of fitting attempts to make per fit, larger number will give more statistically significant results, but will take longer
##### :return: dict {best_fit, (numpy array of final best fit params), final_cost, (float of final cost for the best fit params), time, (float of time taken to generate best fit)}

In [10]:
# fit the maxwellModel instance to the experimental data using Nelder-Mead
# at most, perform 1000 iterations per fit attempt
# the largest possible model will have 3 arms
# the model will be fit sequentially, starting with a 1 arm, then using the best 1 arm parameters as an initial guess
# for the 2 arm model, and the best 2 arm model for the 3 arm model
# perform 10 fit attempts per model
model1.fit(maxiter=1000, max_model_size=3, fit_sequential=True, num_attempts=10)

C:\Users\Windows\PycharmProjects\mat-phys\viscofit.py:164: RuntimeWarning: overflow encountered in exp
  relaxance = - sum(model_params[1::2] / model_params[2::2] * exp(- time_matrix / model_params[2::2]), axis=1)
C:\Users\Windows\PycharmProjects\mat-phys\viscofit.py:164: RuntimeWarning: overflow encountered in multiply
  relaxance = - sum(model_params[1::2] / model_params[2::2] * exp(- time_matrix / model_params[2::2]), axis=1)
C:\Users\Windows\PycharmProjects\mat-phys\viscofit.py:222: RuntimeWarning: overflow encountered in square
  sse = sum((self.LR_force(model_params=model_params) - self.force) ** 2, axis=0)
C:\Users\Windows\PycharmProjects\mat-phys\viscofit.py:158: RuntimeWarning: overflow encountered in exp
  relaxance = - sum(model_params[0::2] / model_params[1::2] * exp(- time_matrix / model_params[1::2]), axis=1)
C:\Users\Windows\PycharmProjects\mat-phys\viscofit.py:158: RuntimeWarning: overflow encountered in multiply
  relaxance = - sum(model_params[0::2] / model_params[1::

{'final_params': array([1.32859268e+05, 9.21688302e+05, 8.95043599e-04, 4.54524461e+04,
        3.96168046e-03]),
 'final_cost': 5.492277694323846e-23,
 'time': 7.997998237609863,
 'trial_variance': 2.6098571947889565e-30}

### maxwellModel.fit_slow(self, maxiter=1000, max_model_size=4, fit_sequential=True, num_attempts=5)

##### fit experimental force distance curve(s) to maxwell model of arbitrary size using simulated annealing with a nelder-mead simplex local search, this is very computationally costly and will take a very long time though typically results in much better fits

##### :param maxiter: int maximum iterations to perform for each fitting attempt (larger number gives longer run time)
##### :param max_model_size: int largest number of arms per maxwell model to test (going larger tends to give poor and unphysical fits)
##### :param fit_sequential: bool whether or not to fit sequentially (cascade fit from previous model as the initial guess of the next) (RECOMMENDED)
##### :param num_attempts: int number of fitting attempts to make per fit, larger number will give more statistically significant results, but will take longer
##### :return: dict {best_fit, (numpy array of final best fit params), final_cost, (float of final cost for the best fit params), time, (float of time taken to generate best fit)}

In [ ]:
# fit the maxwellModel instance to the experimental data using Simulated Annealing with a Nelder-Mead local search
# at most, perform 500 iterations per fit attempt
# the largest possible model will have 3 arms
# the model will not be fit sequentially, fitting all the parameters at once
# perform 7 fit attempts per model
model1.fit_slow(maxiter=500, max_model_size=3, fit_sequential=False, num_attempts=7)

### ----------------
### Kelvin-Voigt Fit
### ----------------

### kelvinVoigtModel(forces, times, indentations, radii, J_logbounds=(-9, -1), T_logbounds=(-5, 0))

##### initializes an instance of the kelvinVoigtModel class used for generating fits, of experimentally obtained force-distance data all belonging to the same sample, to a kelvin-voigt model which corresponds to the sample's viscoelastic behavior
##### :param forces: either list of numpy arrays or single numpy array corresponding to the force signals from an AFM
##### :param times: either list of numpy arrays or single numpy array corresponding to the time signals from an AFM
##### :param indentations: either list of numpy arrays or single numpy array corresponding to the indentation signals from an AFM
##### :param radii: either list of floats or single float corresponding to the tip radii of an AFM
##### :param J_logbounds: tuple (float, float) high and low log bound for the compliance elements in the model
##### :param T_logbounds: tuple (float, float) high and low log bound for the time constants in the model

In [ ]:
# create a kelvinVoigtModel instance
# load the AFM forces, times, indentations, and radii
# define the logbounds for E to be between 4 and 7 -> (10^4, 10^7)
# define the logbounds for T to be between -4 and -2 -> (10^-4, 10^-2)
model1 = vf.maxwellModel(forces=forces, times=times, indentations=indentations, 
                         radii=radii, E_logbounds=(4, 7), T_logbounds=(-4, -2))

# Fit Custom Defined Models

# Generate Maps